In [31]:
import os
import numpy as np
import pandas as pd

In [32]:
assert ("Top_New" in os.listdir(os.getcwd())), "Please create Top_New directory!"

## Getting All Files List

In [33]:
cwd = os.getcwd() + "/"

directory_list = [cwd + "Top_New/"]
file_list = []

while len(directory_list) != 0:

    for dl in directory_list:

        dir_list = next(os.walk(dl))[1]
        for i in range(len(dir_list)):
            dir_list[i] = dl + dir_list[i] + "/"
            
        directory_list.extend(dir_list)

        file_list.extend([dl+fl for fl in next(os.walk(dl))[2]])

        directory_list.remove(dl)

## Extracting Data From The Files

In [34]:
data = {"File_Name": [],
        "Serial": [],
        "Litho_1": [],
        "Litho_2": [],
        "Exam_No": [],
        "Subject_Code": [],
        "Roll": [],
        "Venue_Code": []
       }

split_lenght = [0, 32, 32, 2, 3, 9, 3]
split = np.cumsum(split_lenght)

for fl in file_list:
    current_file = open(fl, "r")
    serial = 1
    for line in current_file:
        data["File_Name"].append(os.path.basename(fl)[:-4])
        data["Serial"].append(f"{serial:010d}")
        
        # iterate through all other keys excpet File_Name and Serial and set their values according to length
        key = list(data.keys())[2:]
        for i in range(0, len(key)):
            if key[i] == "Litho_1" or key[i] == "Litho_2":
                data[key[i]].append(line[split[i]:split[i+1]].replace(" ", "0"))    # replacing <space> with 0 in Litho
            else:
                data[key[i]].append(line[split[i]:split[i+1]])
        serial += 1

In [35]:
df = pd.DataFrame(data)

## Reading Applicants Data

In [36]:
applicants = pd.read_csv("15th_ApplicantsInfo.txt", sep='|', encoding='latin1')

column_names = ["Roll", "Applicants_Subject_Code", "DoB(YMD)", "Invoice", "Subject_Name", "MCQ_Marks", "Applicants_Name", "Post_Code"]
applicants.columns = column_names

## Seperate Invalid Roll

In [37]:
applicants = applicants[["Roll", "Applicants_Subject_Code", "DoB(YMD)", "Applicants_Name"]]

applicants["Roll"] = applicants["Roll"].astype(str)
df["Roll"] = df["Roll"].astype(str)

tdf = pd.merge(df, applicants, on="Roll", how="inner", suffixes=("", "_Main"))

In [38]:
tdf_roll = list(tdf["Roll"])

invalid_roll = df[~df["Roll"].isin(tdf_roll)]

if len(invalid_roll) > 0:
    print("New INVALID ROLL found! Check for Invalid_Roll_Again.csv in Top_Output directory.")
    invalid_roll.to_csv("Top_Output/Invalid_Roll_2.csv", index=False, sep=';')

df = tdf.copy()

## Merging New Tops to Previous Top.csv

In [39]:
previous_top = pd.read_csv("Top_Output/Top.csv", sep=';')

df = pd.concat([previous_top, df])

## Seperating Duplicate Roll, Unmatched Litho and Duplicate Litho Again!

In [40]:
tdf = df[df["Roll"].duplicated(keep=False)]

tdf.to_csv("Top_Output/Duplicate_Roll_2.csv", index=False, sep=';')
df = df.drop(tdf.index)

tdf = df[df["Litho_1"] != df["Litho_2"]]

tdf.to_csv("Top_Output/Unmatched_Litho_Top_2.csv", index=False, sep=';')
df = df.drop(tdf.index)

tdf = df[df.duplicated(["Litho_1", "Litho_2"])]

tdf.to_csv("Top_Output/Duplicate_Top_2.csv", index=False, sep=';')
df = df.drop(tdf.index)

In [41]:
df.to_csv("Top_Output/Top.csv", index=False, sep=';')